### [T&uuml;rme von Hanoi](https://de.wikipedia.org/wiki/T%C3%BCrme_von_Hanoi)
Ein Scheibenstapel soll verschoben werden. Dabei sind folgende Regeln einzuhalten:
- Es darf jeweils nur eine Scheibe verschoben werden.
- Auf eine Scheibe darf nur eine kleinere Scheibe gelegt werden.
- Es d&uuml;rfen max. 3 Stapel vorhanden sein.

Wir programmieren eine einfache, aber ausbauf&auml;hige Variante dieses Spiels.  
- Verschoben wird ein Stabel mit 4 Scheiben,
  repr&auml;sentiert durch die Zahlen 0, 1, 2 und 3. 
  Je gr&ouml;sser die Zahl,
  desto gr&ouml;sser die Scheibe.
- Eine absteigend sortierte Liste `[3, 2, 1, 0]` entspricht einem Stapel.
  Die letzte Zahl entspricht der obersten und kleinsten Scheibe.

### Der Kern des Spiels
Der Spielzustand wird in der Variable `stacks` gespeichert.
Diese enth&auml;lt eine Liste mit 3 Listen, den Stapeln.
Die Funktion `new_game()` startet ein neues Spiel, indem die Liste `stacks` modifiziert wird, so dass sie die 3 Stapel
`[3, 2, 1, 0]`, `[]` und `[]` enth&auml;lt.

Die Funktion `move_disk(src, dst)` verschiebt eine Scheibe von Stapel `src` (source) auf den Stapel `dst` (destination), falls regelkonform.

In [ ]:
ndisks = 4
stacks = []


def new_game():
    stack1 = list(range(ndisks))[::-1]
    stacks[:] = [stack1, [], []]


def move_disk(src, dst):
    # falls stacks[src] leer oder stacks[dst] leer oder Scheibenbed. verletzt, mache nichts
    if not stacks[src] or (stacks[dst] and stacks[dst][-1] < stacks[src][-1]):
        return
    disk = stacks[src].pop()
    stacks[dst].append(disk)

In [ ]:
new_game()
stacks

In [ ]:
move_disk(0, 2)
stacks

In [ ]:
move_disk(0, 1)
stacks

In [ ]:
move_disk(2, 1)
stacks

### Darstellung des Spielzustands an einen Eventhandler delegieren 
Wir k&ouml;nnen bereits ein neues Spiel starten, Scheiben verschieben und dann
die neue Stabelkonfiguration betrachten, indem wir die Liste `stacks` ausgeben.
Wir m&ouml;chten jedoch, dass der Spielzustand automatisch nach einem Neustart des Spiels und nach jeder Stapelverschiebung ausgegeben wird.  

Diese Aufgabe &uuml;bertragen wir einem **Eventhandler**.
Der Eventhandler ist eine Funktion, die mit den Argumenten `event` und `data`
aufgerufen wird. `event` ist dabei der Name des Events und `data` sind die 
f&uuml;r die Bearbeitung des Events relevanten Daten.


Im vorliegenden Fall haben wir 2 Events, Neustart und Stapelverschiebung.
Wir behandeln beide gleich. F&uuml;rs Erste geben wir jeweils eine textliche Darstellung der Stapelkonfiguration aus.

In [ ]:
ndisks = 4
stacks = []


# ein einfacher Eventhandler
event_handler = print


def new_game():
    stack1 = list(range(ndisks))[::-1]
    stacks[:] = [stack1, [], []]
    event_handler('new game', stacks)


def move_disk(src, dst):
    if not stacks[src] or (stacks[dst] and stacks[dst][-1] < stacks[src][-1]):
        return
    disk = stacks[src].pop()
    stacks[dst].append(disk)
    event_handler('update stacks', stacks)

In [ ]:
new_game()

In [ ]:
move_disk(0, 2)

In [ ]:
move_disk(0, 1)

In [ ]:
move_disk(2, 1)

### Bildliche Darstellung der Stapelkonfiguration
Statt einer textlichen Darstellung der Stapelkonfiguration
soll der Eventhandler ein bildliche Darstellung liefern.
Dazu weisen wir der Variable `event_handler` eine Funktion zu, welche
die Scheibenstapel auf eine Canvas zeichnet.


In [ ]:
from IPython.display import display
from widgets_helpers import new_canvas


def draw_stack(canvas, stack, pos):
    h = disk_height
    for i, disk in enumerate(stack):
        canvas.fill_style = colors[disk]
        w = disk_widths[disk]
        canvas.fill_rect(pos - w/2, canvas.height-(i+1)*h, w, h)


def draw_stacks(canvas, event, stacks):
    canvas.clear()
    for pos, stack in zip(stack_positions, stacks):
        draw_stack(canvas, stack, pos)


stack_positions = [50, 150, 250]
disk_widths = [30, 50, 70, 90]
disk_height = 10
colors = ['brown', 'teal', 'blue', 'purple']


event_handler = lambda event, data: draw_stacks(canvas, event, data)
canvas = new_canvas(300, 100)

In [ ]:
canvas

In [ ]:
new_game()

In [ ]:
move_disk(0, 2)

In [ ]:
move_disk(0, 1)

In [ ]:
move_disk(2, 1)

### Spiel mit der Tastatur steuern
Statt Spielstart und Scheibenverschiebung durch manuellen Aufruf der Funktionen
`new_game()` und `move_disk(src, dst)` auszul&ouml;sen, m&ouml;chten wir
dies durch Dr&uuml;cken von `'n'`, bez. durch aufeinanderfolgendes Dr&uuml;cken 
zweier Stapelnummern erreichen. Die Zahlen 1, 2 und 3 stehen f&uuml;r die Stapel 0,1 und 2.

Dazu nutzen wir die F&auml;higkeit des Canvas-Widget auf Tastendr&uuml;cke zu h&ouml;ren.
Ist das Canvas-Widget aktive (passiert wenn man darauf klickt)
und wurde eine Funktion zum bearbeiten des Events `on_key_down` 
mit der Methode `on_key_down` registriert, wird beim Dr&uuml;cken einer Taste
diese Funktion aufgerufen. Das erste Argument ist der Tastennamen.

Zur Bearbeitung des Events `on_key_down` registrieren wir eine
Funktion `on_key_down`. Diese merkt sich in der globalen Variable `src` den Ausgangsstapel. Wird mit dem n&auml;chsten Tastendruck ein Zielstapel `dst` selektiert, so wird  `move_disk(src, dst)` aufgerufen.  

**Bemerkung**:  
W&auml;hrend der Entwicklungsphase m&ouml;chte man die Callbacks editieren und wieder registrieren k&ouml;nnen.
Um zu verhindern, dass mehrere (ev. fehlerhafte) Callbacks f&uuml;r ein Event registriert sind, entfernen wir jeweils alle
registrierten Callbacks, hier f&uuml;r das on_key_down Event mit der Funktion
```python
remove_callbacks('on_key_down', canvas)
```

aus dem modul `canvas_helpers` (File in den Ordner `modules` verschieben!).

In [ ]:
# nur zum Debuggen
from widgets_helpers import new_output
from canvas_helpers import remove_callbacks

In [ ]:
src = None
err_out = new_output()


@err_out.capture()
def on_key_down(key, *flags):
    global src

    if key in '123' and src is None:
        src = int(key) - 1
        return
    elif key in '123':
        dst = int(key) - 1
        move_disk(src, dst)
    if key == 'n':
        new_game()
    src = None


remove_callbacks('on_key_down', canvas)
canvas.on_key_down(on_key_down)

In [ ]:
err_out.clear_output()

In [ ]:
display(canvas, err_out)

### Maussteuerung hinzuf&uuml;gen

In [ ]:
@err_out.capture()
def on_mouse_down(x, y):
    global src
    stack_width = canvas.width / 3
    src = int(x // stack_width)


@err_out.capture()
def on_mouse_up(x, y):
    global src

    if src is None:
        return

    stack_width = canvas.width / 3
    dst = int(x // stack_width)
    move_disk(src, dst)
    src = None


remove_callbacks('on_mouse_down', canvas)
remove_callbacks('on_mouse_up', canvas)
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_up(on_mouse_up)

In [ ]:
err_out.clear_output()

In [ ]:
display(canvas, err_out)